In [ ]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_match_postgresql import *

In [ ]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = ht_time.findLastSunOrWed('2018/04/04')
print("folder =", folder)

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickFacebookLogin, clickFacebookLogout

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick LastKeeper Tab
clickTabOfFireFox(5, isDualMonitor)

# Hattrick Login by Facebook
clickEmpty(410, isDualMonitor)
moveScroll(20)
clickFacebookLogin(484, 395, isDualMonitor)

# # Hattrick Logout by Facebook
# clickEmpty(336, isDualMonitor)
# moveScroll(20)
# clickFacebookLogout(1258, 336, isDualMonitor)

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick LastKeeper Tab
clickTabOfFireFox(5, isDualMonitor)

# Hattrick Login by Facebook
clickEmpty(410, isDualMonitor)
moveScroll(20)
clickFacebookLogin(484, 394, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 340, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(384, 696, 80, 108, isDualMonitor, 'screen-Match.png')

# Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1082, 841, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# SavePageAs
moveAndClick(91, 130, isDualMonitor)

# Type match
time.sleep(1)
typeFileName()

# rwoo folder
moveAndClick(570, 460, isDualMonitor)

# 02_workspace folder
moveAndClicks(720, 440, isDualMonitor, 2)

# 04_Hattrick folder
moveAndClicks(720, 500, isDualMonitor, 2)

# HattrickStudy
moveAndClicks(720, 415, isDualMonitor, 2)

# 00_LastKeeper
moveAndClicks(720, 415, isDualMonitor, 2)

# 2018
moveAndClicks(720, 440, isDualMonitor, 2)

# Last Month
clickLastMonthOrDay()

# Last Day
clickLastMonthOrDay()

# Save
moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# Hattrick Logout by Facebook
clickEmpty(336, isDualMonitor)
moveScroll(20)
clickFacebookLogout(1258, 338, isDualMonitor)

# Jupyter Tab
clickTabOfFireFox(7, isDualMonitor)

In [ ]:
from ht_file import *

In [ ]:
HattrickFile = HattrickFile()
HattrickFile.check_and_delete_folder(folder + '/match_files')

In [ ]:
from ht_match import *

In [ ]:
HattrickMatch = HattrickMatch()
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase')
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [ ]:
import psycopg2

In [ ]:
# def dropMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS match ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
# def createMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE match (
#     Date DATE,          --01
#     Po   VARCHAR(5),    --02 Position
#     Num  INT,           --03 Number
#     RT   NUMERIC(2, 1), --04 Rating
#     sMin INT,           --05 Start Min
#     eMin INT            --06 End Min
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createMatchTable()

In [ ]:
def selectCountMatch():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = "SELECT COUNT(*) FROM match" 
        
        cur.execute(sql)
        row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()
    return row[0]

In [ ]:
selectCountMatch()

In [ ]:
def insertMatchTable(time, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
INSERT INTO match ( 
    SELECT 
        to_date(%s, 'YYYY-MM-DD'), --01 
        %s, %s, %s, %s, %s         --02 03 04 05 06
) ;"""
        cur.execute(sql, ( time,
                           row[0], row[1], row[2], row[3], row[4] ) )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
        print(row)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            insertMatchTable( folder.replace('/','-'), row)

In [ ]:
selectCountMatch()

In [ ]:
def updateRatingOfMatchTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
update match 
    set rt = player.rt 
from player   
where 
    match.date = player.date
    and player.date = player.last
    and match.po = player.po 
    and match.num = player.num 
    and match.rt != player.rt
; """
        cur.execute(sql)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
updateRatingOfMatchTable()

In [ ]:
def getTupleListFromMatch(SelectString, WhereString):
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        OrderByString = ""
        OrderByString += "\n" + "ORDER BY "
        OrderByString += "\n" + "    CASE "
        OrderByString += "\n" + "        WHEN po = 'KP' THEN 1 "
        OrderByString += "\n" + "        WHEN po = 'WB' THEN 2 "
        OrderByString += "\n" + "        WHEN po = 'CD' THEN 3 "
        OrderByString += "\n" + "        WHEN po = 'W'  THEN 4 "
        OrderByString += "\n" + "        WHEN po = 'IM' THEN 5 "
        OrderByString += "\n" + "        WHEN po = 'FW' THEN 6 "
        OrderByString += "\n" + "        ELSE 7 "
        OrderByString += "\n" + "    END "
        
        sql = SelectString + "\n" + "FROM match " + WhereString + OrderByString + "\n"
        print(sql)
        cur.execute(sql)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
SelectString = ""
SelectString += "\n" + "SELECT "
SelectString += "\n" + "    po, num, rt :: VARCHAR(5), smin, emin "

WhereString = ""
WhereString += "\n" + "WHERE "
WhereString += "\n" + "    date = '" + folder.replace('/','-') + "'"

tupleList = getTupleListFromMatch(SelectString, WhereString)
for tuple in tupleList:
    print(tuple[0], tuple[1], tuple[2], tuple[3], tuple[4], sep='\t')

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [ ]:
conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
conn

In [ ]:
hattrickMatchPostgresql = HattrickMatchPostgreSQL()
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'KP', [2,58])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [2,58])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'WB', [6,8,14,52,61,72])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [6,8,14,52,61,72])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'CD', [3,4,38,40,41,70])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [3,4,38,40,41,70])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'W', [11,13,15,23,65])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [11,13,15,23,65])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'IM', [9,10,12,25,34])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [9,10,12,25,34])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'FW', [16,18,31,48,55])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [16,18,31,48,55])

In [ ]:
conn.close()
conn